In [26]:
# Modules
import os
import csv

# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from random import random
from citipy import citipy
from ipyleaflet import Map, basemaps, basemap_to_tiles
import folium

import requests
import time
import json
from pprint import pprint

# Import API key
#from api_keys import api_key

In [27]:
# File to Load ("listings.csv" file)
listing_data = "Data Files for AirBnB Project/listings.csv"

# Set path for "listings.csv" file
listing_data_csvpath = os.path.join("Data Files for AirBnB Project", "listings.csv")
print(listing_data_csvpath)

# Open the "listings.csv"
with open(listing_data_csvpath, newline="") as csvfile:
    listing_data_csv = csv.reader("listings.csv", delimiter=",")

Data Files for AirBnB Project/listings.csv


In [28]:
# Display the "listing.csv" data-table for preview
listing_data_pd = pd.read_csv(listing_data)
listing_data_pd.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,2,155,2019-01-26,1.27,5,300
1,3662,Vita's Hideaway II,4645,Vita,NaN,Historic Anacostia,38.861925,-76.989634,Private room,65,2,35,2018-06-02,0.37,3,340
2,3686,Vita's Hideaway,4645,Vita,NaN,Historic Anacostia,38.863138,-76.988362,Private room,55,2,68,2018-10-27,0.67,3,296
3,3771,Mt. Pleasant,4795,Charlene,NaN,"Columbia Heights, Mt. Pleasant, Pleasant Plain...",38.927605,-77.039265,Private room,88,1,1,2009-01-21,0.01,1,0
4,4002,2 Bedroom Private Condo Suite (Basement Apt),5143,Anthony,NaN,"North Michigan Park, Michigan Park, University...",38.940084,-76.989360,Entire home/apt,120,2,149,2018-11-13,1.22,1,316


In [29]:
# Rename "id" to "listing_id" for merge.
listing_data_pd.rename(columns={'id': 'listing_id'}, inplace=True)
listing_data_pd.head()

,listing_id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,2,155,2019-01-26,1.27,5,300
1,3662,Vita's Hideaway II,4645,Vita,NaN,Historic Anacostia,38.861925,-76.989634,Private room,65,2,35,2018-06-02,0.37,3,340
2,3686,Vita's Hideaway,4645,Vita,NaN,Historic Anacostia,38.863138,-76.988362,Private room,55,2,68,2018-10-27,0.67,3,296
3,3771,Mt. Pleasant,4795,Charlene,NaN,"Columbia Heights, Mt. Pleasant, Pleasant Plain...",38.927605,-77.039265,Private room,88,1,1,2009-01-21,0.01,1,0
4,4002,2 Bedroom Private Condo Suite (Basement Apt),5143,Anthony,NaN,"North Michigan Park, Michigan Park, University...",38.940084,-76.989360,Entire home/apt,120,2,149,2018-11-13,1.22,1,316


In [30]:
# File to Load ("detail listings.csv" file)
detail_listing_data = "Data Files for AirBnB Project/detail listings.csv"

# Set path for "detail listings.csv" file
detail_listing_data_csvpath = os.path.join("Data Files for AirBnB Project", "detail listings.csv")
print(detail_listing_data_csvpath)

# Open the "detail listings.csv"
with open(detail_listing_data_csvpath, newline="") as csvfile:
    detail_listing_data_csv = csv.reader("detail listings.csv", delimiter=",")

Data Files for AirBnB Project/detail listings.csv


In [31]:
# Display the "detail listings.csv" data-table for preview
detail_listing_data_pd = pd.read_csv(detail_listing_data)
detail_listing_data_pd.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3362,https://www.airbnb.com/rooms/3362,20190211014849,2019-02-11,"Convention Center Rowhouse & In Law: 2 Units, 4BR",An architect-designed rowhouse featuring a lar...,This listing features our 19th century Victori...,An architect-designed rowhouse featuring a lar...,none,NaN,...,t,f,strict_14_with_grace_period,f,f,5,5,0,0,1.27
1,3662,https://www.airbnb.com/rooms/3662,20190211014849,2019-02-11,Vita's Hideaway II,IMPORTANT NOTES * Carefully read and be sure t...,IMPORTANT NOTES * Airbnb keeps accurate track ...,IMPORTANT NOTES * Carefully read and be sure t...,none,We love that our neighborhood is up and coming...,...,f,f,moderate,f,f,3,0,3,0,0.37
2,3686,https://www.airbnb.com/rooms/3686,20190211014849,2019-02-11,Vita's Hideaway,IMPORTANT NOTES * Carefully read and be sure t...,IMPORTANT NOTES * Airbnb keeps accurate track ...,IMPORTANT NOTES * Carefully read and be sure t...,none,We love that our neighborhood is up and coming...,...,f,f,moderate,f,f,3,0,3,0,0.67
3,3771,https://www.airbnb.com/rooms/3771,20190211014849,2019-02-11,Mt. Pleasant,"Clean, convenient and welcoming home. Bedroom...","Spacious, clean, convenient and welcoming home...","Clean, convenient and welcoming home. Bedroom...",none,NaN,...,f,f,moderate,f,f,1,0,1,0,0.01
4,4002,https://www.airbnb.com/rooms/4002,20190211014849,2019-02-11,2 Bedroom Private Condo Suite (Basement Apt),This is a very large beautiful English BASEMEN...,This is a very large (1400 SQ feet) beautiful ...,This is a very large beautiful English BASEMEN...,none,This is a very quite neighborhood.,...,t,f,flexible,f,t,1,1,0,0,1.22


In [32]:
# Rename "id" to "listing_id" for merge.
detail_listing_data_pd.rename(columns={'id': 'listing_id'}, inplace=True)
detail_listing_data_pd.head(50)

,listing_id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3362,https://www.airbnb.com/rooms/3362,20190211014849,2019-02-11,"Convention Center Rowhouse & In Law: 2 Units, 4BR",An architect-designed rowhouse featuring a lar...,This listing features our 19th century Victori...,An architect-designed rowhouse featuring a lar...,none,NaN,...,t,f,strict_14_with_grace_period,f,f,5,5,0,0,1.27
1,3662,https://www.airbnb.com/rooms/3662,20190211014849,2019-02-11,Vita's Hideaway II,IMPORTANT NOTES * Carefully read and be sure t...,IMPORTANT NOTES * Airbnb keeps accurate track ...,IMPORTANT NOTES * Carefully read and be sure t...,none,We love that our neighborhood is up and coming...,...,f,f,moderate,f,f,3,0,3,0,0.37
2,3686,https://www.airbnb.com/rooms/3686,20190211014849,2019-02-11,Vita's Hideaway,IMPORTANT NOTES * Carefully read and be sure t...,IMPORTANT NOTES * Airbnb keeps accurate track ...,IMPORTANT NOTES * Carefully read and be sure t...,none,We love that our neighborhood is up and coming...,...,f,f,moderate,f,f,3,0,3,0,0.67
3,3771,https://www.airbnb.com/rooms/3771,20190211014849,2019-02-11,Mt. Pleasant,"Clean, convenient and welcoming home. Bedroom...","Spacious, clean, convenient and welcoming home...","Clean, convenient and welcoming home. Bedroom...",none,NaN,...,f,f,moderate,f,f,1,0,1,0,0.01
4,4002,https://www.airbnb.com/rooms/4002,20190211014849,2019-02-11,2 Bedroom Private Condo Suite (Basement Apt),This is a very large beautiful English BASEMEN...,This is a very large (1400 SQ feet) beautiful ...,This is a very large beautiful English BASEMEN...,none,This is a very quite neighborhood.,...,t,f,flexible,f,t,1,1,0,0,1.22
5,4197,https://www.airbnb.com/rooms/4197,20190211014849,2019-02-11,Bedroom in DC 2 blocks to Metro,This is the middle bedroom upstairs in a resto...,This is a private room in an 1885 rowhouse sui...,This is the middle bedroom upstairs in a resto...,none,"Our area, the Eastern Market neighborhood of C...",...,f,f,moderate,f,f,2,0,2,0,0.31
6,4283,https://www.airbnb.com/rooms/4283,20190211014849,2019-02-11,Adams Morgan Quiet Sunny Lrg 2 Bd w/Porch,Tranquil setting in the middle of one of the m...,"Fully furnished, two bedrooms, full kitchen, s...",Tranquil setting in the middle of one of the m...,none,Within blocks of just about everything you may...,...,f,f,moderate,f,f,1,1,0,0,0.12
7,4501,https://www.airbnb.com/rooms/4501,20190211014849,2019-02-11,DC Rowhouse,We're renting a private bedroom/bathroom in ou...,"Note: we look forward to hosting you, however,...",We're renting a private bedroom/bathroom in ou...,none,"Walking distance to all you need. Metro, great...",...,f,f,strict_14_with_grace_period,f,f,1,0,1,0,0.98
8,4529,https://www.airbnb.com/rooms/4529,20190211014849,2019-02-11,Bertina's House Part One,This is large private bedroom with plenty of...,"A large room with a queen size bed, Cable T.V....",This is large private bedroom with plenty of...,none,Very quiet neighborhood and it is easy accessi...,...,f,f,strict_14_with_grace_period,t,t,3,0,3,0,0.85
9,4967,https://www.airbnb.com/rooms/4967,20190211014849,2019-02-11,"DC, Near Metro",NaN,"Hello, my name is Seveer and I live in a quiet...","Hello, my name is Seveer and I live in a quiet...",none,NaN,...,f,f,moderate,f,f,3,0,3,0,0.31


In [33]:
# File to Load ("detail reviews.csv" file)
detail_reviews_data = "Data Files for AirBnB Project/detail reviews.csv"

# Set path for "detail reviews.csv" file
detail_reviews_data_csvpath = os.path.join("Data Files for AirBnB Project", "detail reviews.csv")
print(detail_reviews_data_csvpath)

# Open the "detail reviews.csv"
with open(detail_reviews_data_csvpath, newline="") as csvfile:
    detail_reviews_data_csv = csv.reader("detail reviews.csv", delimiter=",")

Data Files for AirBnB Project/detail reviews.csv


In [34]:
# Display the "detail reviews.csv" data-table for preview
detail_reviews_data_pd = pd.read_csv(detail_reviews_data)
detail_reviews_data_pd.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3362,662,2009-01-21,6917,Robyn,Ayeh is a gracious and accommodating host. He...
1,3362,1099,2009-03-26,10492,Brian,"I stayed at Ayeh's place in March, and it was ..."
2,3362,6300,2009-07-28,22668,Anthony,"My 14 y.o. niece, her best friend and I stayed..."
3,3362,8387,2009-08-27,30379,Lina,We enjoyed the time with Ayeh in Washington ve...
4,3362,8451,2009-08-28,33059,Nicole,We also enjoyed our stay in DC while at Ayeh's...


In [35]:
# 1st Merge: Combine the listing_data and the detail_listing data-files into a single dataset (merge 2 dataframes using an inner join).
merged_data_1_2_df = pd.merge(listing_data_pd, detail_listing_data_pd, on="listing_id")
merged_data_1_2_df.head()

,listing_id,name_x,host_id_x,host_name_x,neighbourhood_group,neighbourhood_x,latitude_x,longitude_x,room_type_x,price_x,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count_y,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month_y
0,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,t,f,strict_14_with_grace_period,f,f,5,5,0,0,1.27
1,3662,Vita's Hideaway II,4645,Vita,NaN,Historic Anacostia,38.861925,-76.989634,Private room,65,...,f,f,moderate,f,f,3,0,3,0,0.37
2,3686,Vita's Hideaway,4645,Vita,NaN,Historic Anacostia,38.863138,-76.988362,Private room,55,...,f,f,moderate,f,f,3,0,3,0,0.67
3,3771,Mt. Pleasant,4795,Charlene,NaN,"Columbia Heights, Mt. Pleasant, Pleasant Plain...",38.927605,-77.039265,Private room,88,...,f,f,moderate,f,f,1,0,1,0,0.01
4,4002,2 Bedroom Private Condo Suite (Basement Apt),5143,Anthony,NaN,"North Michigan Park, Michigan Park, University...",38.940084,-76.989360,Entire home/apt,120,...,t,f,flexible,f,t,1,1,0,0,1.22


In [36]:
# 2nd Merge: Combine the merged_data_1_2 and the detail_reviews data-files into a single dataset.
merged_data_1_2_3_df = pd.merge(merged_data_1_2_df, detail_reviews_data_pd, on="listing_id")
merged_data_1_2_3_df.head(50)

,listing_id,name_x,host_id_x,host_name_x,neighbourhood_group,neighbourhood_x,latitude_x,longitude_x,room_type_x,price_x,...,calculated_host_listings_count_y,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month_y,id,date,reviewer_id,reviewer_name,comments
0,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,662,2009-01-21,6917,Robyn,Ayeh is a gracious and accommodating host. He...
1,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,1099,2009-03-26,10492,Brian,"I stayed at Ayeh's place in March, and it was ..."
2,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,6300,2009-07-28,22668,Anthony,"My 14 y.o. niece, her best friend and I stayed..."
3,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,8387,2009-08-27,30379,Lina,We enjoyed the time with Ayeh in Washington ve...
4,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,8451,2009-08-28,33059,Nicole,We also enjoyed our stay in DC while at Ayeh's...
5,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,9807,2009-09-12,28388,Laura,Ayeh and her apartment were absolutely fantast...
6,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,12271,2009-10-03,39883,Rebecca,It was an absolute pleasure staying with Ayeh....
7,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,12693,2009-10-06,41272,Sophia,"Ayeh was a great host, very informative and kn..."
8,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,13533,2009-10-13,37882,Alec,Ayeh was incredibly courteous and her place wa...
9,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,14779,2009-10-24,42024,Sandy,Ayeh and Ralph were terrific. The first eveni...


In [37]:
# Data Cleanup: Rename "latitude_x" to "latitude" and "longitude_x" to "longitude".
merged_data_1_2_3_df.rename(columns={'latitude_x': 'latitude', "longitude_x": "longitude"}, inplace=True)
merged_data_1_2_3_df.head(50)

,listing_id,name_x,host_id_x,host_name_x,neighbourhood_group,neighbourhood_x,latitude,longitude,room_type_x,price_x,...,calculated_host_listings_count_y,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month_y,id,date,reviewer_id,reviewer_name,comments
0,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,662,2009-01-21,6917,Robyn,Ayeh is a gracious and accommodating host. He...
1,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,1099,2009-03-26,10492,Brian,"I stayed at Ayeh's place in March, and it was ..."
2,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,6300,2009-07-28,22668,Anthony,"My 14 y.o. niece, her best friend and I stayed..."
3,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,8387,2009-08-27,30379,Lina,We enjoyed the time with Ayeh in Washington ve...
4,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,8451,2009-08-28,33059,Nicole,We also enjoyed our stay in DC while at Ayeh's...
5,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,9807,2009-09-12,28388,Laura,Ayeh and her apartment were absolutely fantast...
6,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,12271,2009-10-03,39883,Rebecca,It was an absolute pleasure staying with Ayeh....
7,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,12693,2009-10-06,41272,Sophia,"Ayeh was a great host, very informative and kn..."
8,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,13533,2009-10-13,37882,Alec,Ayeh was incredibly courteous and her place wa...
9,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,Ayeh,NaN,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,...,5,5,0,0,1.27,14779,2009-10-24,42024,Sandy,Ayeh and Ralph were terrific. The first eveni...


In [38]:
# Identify how many unique listing_id there are.
merged_data_1_2_3_df.listing_id.nunique()

7094

In [39]:
# Delete uninteresting columns (must do first before deleting duplicate listing_id rows).
column_drop_df = merged_data_1_2_3_df.drop(columns=['host_name_x', 'neighbourhood_group', 'calculated_host_listings_count_y', 
                                   'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
                                  'calculated_host_listings_count_shared_rooms', 'id', 'date', 'reviewer_id', 
                                   'reviewer_name', 'comments', 'requires_license', 'license', 'jurisdiction_names',
                                  'instant_bookable', 'is_business_travel_ready', 'cancellation_policy', 'require_guest_profile_picture',
                                  'require_guest_phone_verification', 'minimum_nights_x', 'first_review', 'last_review_y', 
                                                    'number_of_reviews_y', 'last_review_x'])
column_drop_df.head()

# Delete ROW duplicates of same listing_id (row duplicates exitst because of number of unique reviews for the same listing_id.). 
row_drop_df = column_drop_df.drop_duplicates(subset='listing_id', keep='first')
row_drop_df.head(50)

,listing_id,name_x,host_id_x,neighbourhood_x,latitude,longitude,room_type_x,price_x,number_of_reviews_x,reviews_per_month_x,...,calendar_last_scraped,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month_y
0,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,155,1.27,...,2019-02-11,26,96.0,10.0,9.0,10.0,10.0,10.0,10.0,1.27
155,3662,Vita's Hideaway II,4645,Historic Anacostia,38.861925,-76.989634,Private room,65,35,0.37,...,2019-02-11,1,94.0,9.0,9.0,10.0,10.0,8.0,9.0,0.37
190,3686,Vita's Hideaway,4645,Historic Anacostia,38.863138,-76.988362,Private room,55,68,0.67,...,2019-02-11,1,91.0,9.0,9.0,10.0,10.0,7.0,9.0,0.67
258,3771,Mt. Pleasant,4795,"Columbia Heights, Mt. Pleasant, Pleasant Plain...",38.927605,-77.039265,Private room,88,1,0.01,...,2019-02-11,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01
259,4002,2 Bedroom Private Condo Suite (Basement Apt),5143,"North Michigan Park, Michigan Park, University...",38.940084,-76.989360,Entire home/apt,120,149,1.22,...,2019-02-11,9,85.0,9.0,8.0,9.0,9.0,8.0,8.0,1.22
408,4197,Bedroom in DC 2 blocks to Metro,5061,"Capitol Hill, Lincoln Park",38.887906,-76.996678,Private room,83,37,0.31,...,2019-02-11,15,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.31
445,4283,Adams Morgan Quiet Sunny Lrg 2 Bd w/Porch,5498,"Kalorama Heights, Adams Morgan, Lanier Heights",38.922032,-77.044474,Entire home/apt,135,13,0.12,...,2019-02-11,2,100.0,10.0,10.0,10.0,10.0,10.0,9.0,0.12
458,4501,DC Rowhouse,1585,"Shaw, Logan Circle",38.913307,-77.024358,Private room,475,120,0.98,...,2019-02-11,0,98.0,10.0,10.0,10.0,10.0,9.0,10.0,0.98
578,4529,Bertina's House Part One,5803,"Eastland Gardens, Kenilworth",38.906449,-76.943050,Private room,52,99,0.85,...,2019-02-11,3,93.0,10.0,9.0,10.0,10.0,9.0,10.0,0.85
677,4967,"DC, Near Metro",7086,"Ivy City, Arboretum, Trinidad, Carver Langston",38.912626,-76.992206,Private room,99,31,0.31,...,2019-02-11,0,95.0,9.0,10.0,10.0,10.0,8.0,9.0,0.31


In [40]:
# Rename "neighbourhood_x" to "neighbourhood"
row_drop_df.rename(columns={'neighbourhood_x': 'neighbourhood'}, inplace=True)
row_drop_df.head(50)

,listing_id,name_x,host_id_x,neighbourhood,latitude,longitude,room_type_x,price_x,number_of_reviews_x,reviews_per_month_x,...,calendar_last_scraped,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month_y
0,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,155,1.27,...,2019-02-11,26,96.0,10.0,9.0,10.0,10.0,10.0,10.0,1.27
155,3662,Vita's Hideaway II,4645,Historic Anacostia,38.861925,-76.989634,Private room,65,35,0.37,...,2019-02-11,1,94.0,9.0,9.0,10.0,10.0,8.0,9.0,0.37
190,3686,Vita's Hideaway,4645,Historic Anacostia,38.863138,-76.988362,Private room,55,68,0.67,...,2019-02-11,1,91.0,9.0,9.0,10.0,10.0,7.0,9.0,0.67
258,3771,Mt. Pleasant,4795,"Columbia Heights, Mt. Pleasant, Pleasant Plain...",38.927605,-77.039265,Private room,88,1,0.01,...,2019-02-11,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01
259,4002,2 Bedroom Private Condo Suite (Basement Apt),5143,"North Michigan Park, Michigan Park, University...",38.940084,-76.989360,Entire home/apt,120,149,1.22,...,2019-02-11,9,85.0,9.0,8.0,9.0,9.0,8.0,8.0,1.22
408,4197,Bedroom in DC 2 blocks to Metro,5061,"Capitol Hill, Lincoln Park",38.887906,-76.996678,Private room,83,37,0.31,...,2019-02-11,15,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.31
445,4283,Adams Morgan Quiet Sunny Lrg 2 Bd w/Porch,5498,"Kalorama Heights, Adams Morgan, Lanier Heights",38.922032,-77.044474,Entire home/apt,135,13,0.12,...,2019-02-11,2,100.0,10.0,10.0,10.0,10.0,10.0,9.0,0.12
458,4501,DC Rowhouse,1585,"Shaw, Logan Circle",38.913307,-77.024358,Private room,475,120,0.98,...,2019-02-11,0,98.0,10.0,10.0,10.0,10.0,9.0,10.0,0.98
578,4529,Bertina's House Part One,5803,"Eastland Gardens, Kenilworth",38.906449,-76.943050,Private room,52,99,0.85,...,2019-02-11,3,93.0,10.0,9.0,10.0,10.0,9.0,10.0,0.85
677,4967,"DC, Near Metro",7086,"Ivy City, Arboretum, Trinidad, Carver Langston",38.912626,-76.992206,Private room,99,31,0.31,...,2019-02-11,0,95.0,9.0,10.0,10.0,10.0,8.0,9.0,0.31


In [41]:
# Reset index.
row_drop_df = row_drop_df.reset_index(drop=True)
row_drop_df.head(50)

,listing_id,name_x,host_id_x,neighbourhood,latitude,longitude,room_type_x,price_x,number_of_reviews_x,reviews_per_month_x,...,calendar_last_scraped,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month_y
0,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,155,1.27,...,2019-02-11,26,96.0,10.0,9.0,10.0,10.0,10.0,10.0,1.27
1,3662,Vita's Hideaway II,4645,Historic Anacostia,38.861925,-76.989634,Private room,65,35,0.37,...,2019-02-11,1,94.0,9.0,9.0,10.0,10.0,8.0,9.0,0.37
2,3686,Vita's Hideaway,4645,Historic Anacostia,38.863138,-76.988362,Private room,55,68,0.67,...,2019-02-11,1,91.0,9.0,9.0,10.0,10.0,7.0,9.0,0.67
3,3771,Mt. Pleasant,4795,"Columbia Heights, Mt. Pleasant, Pleasant Plain...",38.927605,-77.039265,Private room,88,1,0.01,...,2019-02-11,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01
4,4002,2 Bedroom Private Condo Suite (Basement Apt),5143,"North Michigan Park, Michigan Park, University...",38.940084,-76.989360,Entire home/apt,120,149,1.22,...,2019-02-11,9,85.0,9.0,8.0,9.0,9.0,8.0,8.0,1.22
5,4197,Bedroom in DC 2 blocks to Metro,5061,"Capitol Hill, Lincoln Park",38.887906,-76.996678,Private room,83,37,0.31,...,2019-02-11,15,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.31
6,4283,Adams Morgan Quiet Sunny Lrg 2 Bd w/Porch,5498,"Kalorama Heights, Adams Morgan, Lanier Heights",38.922032,-77.044474,Entire home/apt,135,13,0.12,...,2019-02-11,2,100.0,10.0,10.0,10.0,10.0,10.0,9.0,0.12
7,4501,DC Rowhouse,1585,"Shaw, Logan Circle",38.913307,-77.024358,Private room,475,120,0.98,...,2019-02-11,0,98.0,10.0,10.0,10.0,10.0,9.0,10.0,0.98
8,4529,Bertina's House Part One,5803,"Eastland Gardens, Kenilworth",38.906449,-76.943050,Private room,52,99,0.85,...,2019-02-11,3,93.0,10.0,9.0,10.0,10.0,9.0,10.0,0.85
9,4967,"DC, Near Metro",7086,"Ivy City, Arboretum, Trinidad, Carver Langston",38.912626,-76.992206,Private room,99,31,0.31,...,2019-02-11,0,95.0,9.0,10.0,10.0,10.0,8.0,9.0,0.31


In [42]:
# GroupBy "listing_id" and "neighbourhood"
grouped_df = row_drop_df.groupby(['neighbourhood'])
grouped_df.head(50)

,listing_id,name_x,host_id_x,neighbourhood,latitude,longitude,room_type_x,price_x,number_of_reviews_x,reviews_per_month_x,...,calendar_last_scraped,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month_y
0,3362,"Convention Center Rowhouse & In Law: 2 Units, 4BR",2798,"Shaw, Logan Circle",38.910461,-77.019331,Entire home/apt,433,155,1.27,...,2019-02-11,26,96.0,10.0,9.0,10.0,10.0,10.0,10.0,1.27
1,3662,Vita's Hideaway II,4645,Historic Anacostia,38.861925,-76.989634,Private room,65,35,0.37,...,2019-02-11,1,94.0,9.0,9.0,10.0,10.0,8.0,9.0,0.37
2,3686,Vita's Hideaway,4645,Historic Anacostia,38.863138,-76.988362,Private room,55,68,0.67,...,2019-02-11,1,91.0,9.0,9.0,10.0,10.0,7.0,9.0,0.67
3,3771,Mt. Pleasant,4795,"Columbia Heights, Mt. Pleasant, Pleasant Plain...",38.927605,-77.039265,Private room,88,1,0.01,...,2019-02-11,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01
4,4002,2 Bedroom Private Condo Suite (Basement Apt),5143,"North Michigan Park, Michigan Park, University...",38.940084,-76.989360,Entire home/apt,120,149,1.22,...,2019-02-11,9,85.0,9.0,8.0,9.0,9.0,8.0,8.0,1.22
5,4197,Bedroom in DC 2 blocks to Metro,5061,"Capitol Hill, Lincoln Park",38.887906,-76.996678,Private room,83,37,0.31,...,2019-02-11,15,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0.31
6,4283,Adams Morgan Quiet Sunny Lrg 2 Bd w/Porch,5498,"Kalorama Heights, Adams Morgan, Lanier Heights",38.922032,-77.044474,Entire home/apt,135,13,0.12,...,2019-02-11,2,100.0,10.0,10.0,10.0,10.0,10.0,9.0,0.12
7,4501,DC Rowhouse,1585,"Shaw, Logan Circle",38.913307,-77.024358,Private room,475,120,0.98,...,2019-02-11,0,98.0,10.0,10.0,10.0,10.0,9.0,10.0,0.98
8,4529,Bertina's House Part One,5803,"Eastland Gardens, Kenilworth",38.906449,-76.943050,Private room,52,99,0.85,...,2019-02-11,3,93.0,10.0,9.0,10.0,10.0,9.0,10.0,0.85
9,4967,"DC, Near Metro",7086,"Ivy City, Arboretum, Trinidad, Carver Langston",38.912626,-76.992206,Private room,99,31,0.31,...,2019-02-11,0,95.0,9.0,10.0,10.0,10.0,8.0,9.0,0.31


In [43]:
# Identify how many unique listing_id there are.
row_drop_df.listing_id.nunique()

7094

In [44]:
DC_coordinates = (38.9072, -77.0369)
# Create empty map zoomed in on Washington, D.C.
map = folium.Map(location=DC_coordinates, zoom_start=12)

# Added Points of Interest to the DC map
folium.Marker([38.889931, -77.009003], popup='US Capital Building').add_to(map)
folium.Marker([38.8893, -77.0502], popup='Lincoln Memorial').add_to(map)
folium.Marker([38.9072, -77.0369], popup='Washington Monument').add_to(map)
folium.Marker([38.8977, -77.0365], popup='The White House').add_to(map)
folium.Marker([38.8896, -77.0230], popup='National Mall').add_to(map)
folium.Marker([38.8814, -77.0365], popup='Thomas Jefferson Memorial').add_to(map)
folium.Marker([38.8783, -77.0687], popup='Arlington National Cemetery').add_to(map)
folium.Marker([38.9296, -77.0498], popup='Smithsonian National Zoological Park').add_to(map)
folium.Marker([38.8719, -77.0563], popup='The Pentagon').add_to(map)

display(map)

In [45]:
#map.save('DC_coordinates_with_markers.html')